In [3]:
import sys
from os import path
from os import walk

import re

import pandas as pd
import json
import logging
import pandas_profiling
from pandas.io.json import json_normalize 

## Get Columns

In [85]:
with open("../data/fhir-json/2rows/Patient/1.Patient.json") as f:
    d = json.load(f)

awhcuris = json_normalize(d)
address_data = awhcuris.head(31)
address_data.T

data_list = list(address_data)
data_list

['address',
 'birthDate',
 'communication',
 'deceasedDateTime',
 'gender',
 'id',
 'identifier',
 'maritalStatus.coding',
 'maritalStatus.text',
 'multipleBirthBoolean',
 'name',
 'resourceType',
 'telecom',
 'text.div',
 'text.status']

## Flatten

In [ ]:
def flattenDict(d, result=None):
    
    if result is None:
        result = {}
        
    for key, value in list(d.items()):
        value = d[key]
        
        #TODO: OPtimiza strip clean
        if isinstance(value, str):
            value = value.replace("'", ' ')
                
        if isinstance(value, dict):
            #print('THIS IS DICT: ========')
            #print(value)
            value1 = {}
            for keyIn in value:
                #print('DICT keyIn :', keyIn)
                value1[".".join([key,keyIn])]=value[keyIn.lower()]
            flattenDict(value1, result)
        elif isinstance(value, (list, tuple)):  
            #print('THIS IS TUPEL: ', value)
            for indexB, element in enumerate(value):
                if isinstance(element, dict):
                    value1 = {}
                    index = 0
                    for keyIn in element:
                        #print('TUPLES keyIn :', keyIn)
                        #print('key :', key)
                        newkey = ".".join([key,keyIn])        
                        value1[".".join([key,keyIn])]=value[indexB][keyIn]
                        index += 1
                    for keyA in value1:
                        flattenDict(value1, result)   
        else: 
            result[key.lower()]=value
            
            #columns.append(key.lower())
            #print('[key]: ', key)
            
            
            #print('result[key]: ', result[key])
            
    
    return result

In [5]:
class MakeDirectory:
    
    def __init__(self, path):
        self.path = path

    def create_dir(self, folders=None):
        dirName = ''
        
        if isinstance(folders, list):
            
            for folder in folders:
                dirName =  self.path + '/' + folder
                try:
                    os.makedirs(dirName)
                    print("Directory " , dirName ,  " Created ") 
                except FileExistsError:
                    print("Directory " , dirName ,  " already exists")
        else:
            os.makedirs(dirName)

## Folder directory Iterator

In [115]:
class FileIterator:
    
    def __init__(self, input_root, input_folders):
        self.input_data_dir = input_root
        self.input_folders = input_folders
        
    def iterate_filenames(self):
        resources = self.__iterate_dirfiles()
        
        files = self.__iterate_file(resources)
        
        return files
        
    def __iterate_dirfiles(self):
        path_files = []
        file_names = self.input_folders

        for file in file_names:
            path_files.append(input_data_dir + '/' + file)
        return path_files
    
    def __iterate_file(self, path_dbs):
        path_db_workload_files = []     
        for path_db in path_dbs:
            for root, dirs, files in walk(path_db):  
                for filename in files:
                    if not filename.endswith(('.json~', '.swp', '-checkpoint.json')):
                        print('valid file: ',filename)
                        path_db_workload_files.append(root + '/' + filename)
                    if ''.join(dirs) != '.ipynb_checkpoints':
                        pass

        return path_db_workload_files

## Filecontent reader

In [9]:
class ContentReader():
    
    def __init__(self, filename_paths):
        self.filename_paths = filename_paths
        
    def get_data(self):
        data = []
        for filename_path in self.filename_paths:
            
            json = ReadJSON()
            data.append(json.read(filename_path))

        return data

## File Open

In [10]:
class ReadJSON():
    
    @staticmethod
    def read(file_path):
        with open(file_path) as f:
            d = json.load(f)
        return flattenDict(d)

## Print Flatten FHIR JSON

In [82]:
class MakeResult():
    def __init__(self, data, source_paths, destination_path):
        self.source_paths = source_paths
        self.data = data
        self.destination_path = destination_path
        
    def outputs(self):      
        for resource in self.data:
           
            destination_path = self.destination_path + '/' + resource['resourcetype']
            #print('destination: ' ,destination_path)
            id = resource['id'].split('-')[0]
     
            f = open(destination_path + '/' + id + '.' + resource['resourcetype'] + '.json', 'w')
            f.write(json.dumps(resource))
            f.close()
        return

In [151]:
root_dir = '../'

input_data_folders = ['Patient']
input_data_dir = root_dir + 'output/fhir-json/100'

output_directory = root_dir + 'output/fhir-json-cassandra/100'

directory = MakeDirectory(output_directory)
directory.create_dir(input_data_folders)
    
files = FileIterator(input_data_dir, input_data_folders)
input_dirpaths = files.iterate_filenames()
input_dirpaths

data = []
files = ContentReader(input_dirpaths)
data = files.get_data()

result = MakeResult(data, input_dirpaths, output_directory)
result.outputs()

#TODO OPTIMIZE array 
#TODO OPTIMIZE DYNAMIC FIELDS DDL GENERATOR

Directory  ../output/fhir-json-cassandra/100/Patient  already exists
valid file:  0.Patient.json
valid file:  1.Patient.json
valid file:  10.Patient.json
valid file:  11.Patient.json
valid file:  12.Patient.json
valid file:  13.Patient.json
valid file:  14.Patient.json
valid file:  15.Patient.json
valid file:  16.Patient.json
valid file:  17.Patient.json
valid file:  18.Patient.json
valid file:  19.Patient.json
valid file:  2.Patient.json
valid file:  20.Patient.json
valid file:  21.Patient.json
valid file:  22.Patient.json
valid file:  23.Patient.json
valid file:  24.Patient.json
valid file:  25.Patient.json
valid file:  26.Patient.json
valid file:  27.Patient.json
valid file:  28.Patient.json
valid file:  29.Patient.json
valid file:  3.Patient.json
valid file:  30.Patient.json
valid file:  31.Patient.json
valid file:  32.Patient.json
valid file:  33.Patient.json
valid file:  34.Patient.json
valid file:  35.Patient.json
valid file:  36.Patient.json
valid file:  37.Patient.json
valid f